In [2]:
import os
import sys

PROJECT_ROOT = '/'

sys.path.append(PROJECT_ROOT)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [3]:
conf = {'groups_desc': 'data/small_group_description.csv',
    'train_x': 'data/transactions_train.csv',
    'test_x': 'data/transactions_test.csv',
    'train_target': 'data/train_target.csv',
    'test_target': 'data/test.csv'}

In [4]:
from data_utils import *

In [5]:
%%time

ds = DataSource(conf)
ds.read_data()
ds.validation_split()

Reading groups_desc...
Reading train_x...
Reading test_x...
Reading train_target...
Reading test_target...
Validation split: by clientID
CPU times: user 12.8 s, sys: 6.13 s, total: 18.9 s
Wall time: 22.5 s


In [83]:
print(ds)

Dataset: groups_desc, shape: (204, 2)
Dataset: train_x, shape: (21295142, 4)
Dataset: test_x, shape: (17667328, 4)
Dataset: train_target, shape: (24145, 1)
Dataset: test_target, shape: (20000, 1)
Dataset: valid_x, shape: (5155435, 4)
Dataset: valid_target, shape: (5855, 2)


In [6]:
train_target = ds.get_data('train_target')
train_target.set_index('client_id', inplace=True)

In [200]:
ds.get_data('groups_desc')[150:]

,small_group,small_group_code
150,Антиквариат,150
151,Строительные подрядчики,151
152,Услуги химчистки (ковры),152
153,Велоспорт (покупка велосипеда или комплектующих),153
154,Автодилеры (мотоциклы),154
155,Бассейны,155
156,Электромонтажные работы и оборудование (свет),156
157,Электромонтажные работы и оборудование (сигнал...,157
158,Похоронные услуги,158
159,Кемпинг (оборудование),159


In [7]:
train_x = ds.get_data('train_x')
print(train_x.head())


   client_id  trans_date  small_group  amount_rur
0       3694          26            1      10.006
1       3694          29            3      54.955
2       3694          31            1      10.945
3       3694          34           36      61.721
4       3694          36           25       4.579


In [203]:
gr6 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==169).sum()}))

#gr.set_index('client_id', inplace=True)
joined = train_target.join(gr6)
zero =  joined.groupby('bins').agg({'s6':'mean'})
print(zero.head())

            s6
bins          
0     0.015202
1     0.007810
2     0.000329
3     0.017969


In [8]:
train_x['month_number'] = train_x['trans_date']//30

In [9]:
len(train_x.groupby(['client_id']))

24145

In [10]:
#print(train_target.head())
print(train_target.join(train_x))
#train_x.groupby(['client_id'])["trans_date"].agg({"tr_c": "count"}).tr_c.max()


           bins  client_id  trans_date  small_group  amount_rur  month_number
client_id                                                                    
1046          0       3694         710           80      30.362            23
34089         2      34445         669            9     101.208            22
34848         1      32728         502           24     665.480            16
47076         3      19350          48           11      22.486             1
10938         2       6767         593            1      82.963            19
...         ...        ...         ...          ...         ...           ...
36716         2      13026         709           24     107.086            23
14303         1      38817         535            3      21.564            17
22301         2      31317         165            1      16.211             5
25731         0      46358         561            1      10.199            18
16820         3       4323         506            1      60.258 

In [135]:
gr1 = train_x.groupby(['client_id'])["amount_rur"].agg({"s1": "sum"})
print(gr1.head())
#gr.set_index('client_id')
joined = train_target.join(gr1)
zero =  joined.groupby('bins').agg({'s1':'mean'})
print(zero.head())
#print(joined[joined["client_id"]==3694].head(20))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


                  s1
client_id           
4          28404.121
6          15720.739
10         34419.365
11         26789.404
13         17337.467
                s1
bins              
0     46951.066337
1     62098.233179
2     28172.588712
3     60398.480530


In [136]:
gr2 = train_x.groupby(['client_id'])["amount_rur"].agg({"s2": "max"})
joined = train_target.join(gr2)
zero =  joined.groupby('bins').agg({'s2':'mean'})
print(zero.head())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


               s2
bins             
0     2589.771139
1     3578.836228
2     1581.059080
3     3319.801815


In [13]:
gr3 = train_x.groupby(['client_id'])["amount_rur"].agg({"s3": "min"})
joined = train_target.join(gr3)
zero =  joined.groupby('bins').agg({'s3':'mean'})
print(zero.head())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


            s3
bins          
0     0.294786
1     0.557962
2     0.196993
3     0.396317


In [17]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'c4':'min'})
gr4  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['c4'].idxmax()][['client_id','small_group']]
gr4.columns = ['client_id', 'c4']
gr4.set_index('client_id', inplace=True)

joined = train_target.join(gr4)
zero =  joined.groupby('bins').agg({'c4':'mean'})
print(zero.head())

             c4
bins           
0     47.302038
1     47.874875
2     44.124609
3     48.889878


In [18]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'c5':'sum'})
gr5  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['c5'].idxmax()][['client_id','small_group']]
gr5.columns = ['client_id', 'c5']
gr5.set_index('client_id', inplace=True)

joined = train_target.join(gr5)
zero =  joined.groupby('bins').agg({'c5':'mean'})
print(zero.head())

            c5
bins          
0     7.071834
1     5.203390
2     7.260412
3     6.230630


In [137]:
gr6 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==4).sum()}))

#gr.set_index('client_id', inplace=True)
joined = train_target.join(gr6)
zero =  joined.groupby('bins').agg({'s6':'mean'})
print(zero.head())

             s6
bins           
0     36.456231
1     53.253074
2     29.438519
3     41.931091


In [9]:
gr11 = train_x.groupby(['client_id'])["trans_date"].agg({"s11": "count"})
joined = train_target.join(gr11)
zero =  joined.groupby('bins').agg({'s11':'mean'})
print(zero.head())

             s11
bins            
0     892.768460
1     859.832669
2     889.435226
3     885.795912


In [16]:
grouped = train_x.groupby(['client_id', 'month_number'], as_index=False)

In [100]:

client_per_month_spendings = train_x.groupby(['client_id', 'month_number']).agg({"amount_rur": "sum"}).groupby('client_id').agg({'amount_rur':'mean'})

In [102]:
grouped_by_group = train_x.groupby(['client_id', 'small_group'], as_index = False)['small_group'].agg({'freq':'count'})
client_per_max_frequent_category  = grouped_by_group.loc[grouped_by_group.groupby('client_id')['freq'].idxmax()][['client_id','small_group']]
client_per_max_frequent_category.columns = ['client_id', 'freq_group']
client_per_max_frequent_category.set_index('client_id', inplace=True)
print(client_per_max_frequent_category.head())


           freq_group
client_id            
4                   1
6                   1
10                  1
11                  1
13                 11


In [181]:
# Вычислием среднюю максимальную транзакцию по клиентам. Далее найдем среднее отклонение максим транзакций по клиентам. 
# Это позволит найти "большие" транзакции , как mean + std. у нас не гаусово распределение, но все же оперделим значение "большой" транзакции 
# именно так.
grouped_by_max = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'max'})
print(grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] )
large_transactions = grouped_by_max.agg({'amount_rur':'mean'})['amount_rur'] /5
print(large_transactions)

2765.900111078898
553.1800222157797


In [182]:
len(train_x[train_x['amount_rur'] > large_transactions]['client_id'].unique())

20288

In [189]:
grouped_by_min = train_x.groupby(['client_id'], as_index = False).agg({'amount_rur':'min'})
small_transactions = grouped_by_min.agg({'amount_rur':'mean'})['amount_rur'] + grouped_by_min.agg({'amount_rur':'std'})['amount_rur']
print(small_transactions)

0.8787477259259386


In [190]:
len(train_x[train_x['amount_rur'] < small_transactions]['client_id'].unique())

21109

In [191]:
client_per_small_count = train_x[train_x['amount_rur'] < small_transactions].groupby(['client_id'])['trans_date'].agg({'small_num':'count'})

In [203]:
client_per_large_count = train_x[train_x['amount_rur'] > large_transactions].groupby(['client_id'])['trans_date'].agg({'large_num':'count'})

In [109]:
clean_trans_per_month = train_x.groupby(['client_id', 'month_number'])['amount_rur'].agg({"tran_num_mean": "count"}).groupby('client_id').agg({'tran_num_mean':'mean'})

In [193]:
client_max = train_x.groupby(['client_id'])['amount_rur'].agg({"max": "max"})
print(client_max.head())

                max
client_id          
4          1341.802
6           315.781
10          654.893
11         2105.058
13         1190.322


In [204]:
print(client_per_month_spendings.shape)
print(client_per_max_frequent_category.shape)
print(clean_trans_per_month.shape)
print(client_per_small_count.shape)
print(client_per_large_count.shape)

(24145, 1)
(24145, 1)
(24145, 1)
(21109, 1)
(20288, 1)


In [19]:
print(len(gr1))
res_df = gr1.join(gr2).join(gr3).join(gr4).join(gr5)
res_df.fillna(0, inplace=True)


print(res_df.head(10))


24145
                  s1        s2     s3  c4  c5
client_id                                    
4          28404.121  1341.802  0.043   2   1
6          15720.739   315.781  0.045  80   1
10         34419.365   654.893  0.045  23   1
11         26789.404  2105.058  0.388  66  88
13         17337.467  1190.322  0.043  83  11
14         26622.629  1167.291  0.043   5   1
17         58337.998  5692.843  0.028  54   1
19         46941.495  2049.338  0.259  26   1
20         79997.006  3213.706  0.043  73  13
25         77245.103  2564.078  0.647  65   1


In [20]:
from sklearn import preprocessing

column_names_to_normalize = ["amount_rur", "max", "tran_num_mean", "small_num", "large_num"]
x = res_df[column_names_to_normalize].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = res_df.index)
res_df[column_names_to_normalize] = df_temp

print(res_df.head(30))

KeyError: "None of [Index(['amount_rur', 'max', 'tran_num_mean', 'small_num', 'large_num'], dtype='object')] are in the [columns]"

In [26]:
train_dataset = Ds(data = res_df, labels = train_target)
print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)  

NameError: name 'Ds' is not defined

In [159]:
def prepare_featured_data_v2(df):
    gr_by_client = df.groupby(['client_id'])
    gr1 = gr_by_client["amount_rur"].agg({"s1": "sum"})
    g
    r2 = gr_by_client["amount_rur"].agg({"s2": "max"}) 
    gr3 = gr_by_client["amount_rur"].agg({"s3": "min"})

    gr4_by_group = df.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'s4':'min'})
    gr4  = gr4_by_group.loc[gr4_by_group.groupby('client_id')['s4'].idxmax()][['client_id','small_group']]
    gr4.columns = ['client_id', 's4']
    gr4.set_index('client_id', inplace=True)

    
    gr5_by_group = df.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'s5':'sum'})
    gr5  = gr5_by_group.loc[gr5_by_group.groupby('client_id')['s5'].idxmax()][['client_id','small_group']]
    gr5.columns = ['client_id', 's5']
    gr5.set_index('client_id', inplace=True)
    
    gr6 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==4).sum() + 0.000001}))

    gr7 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s7':
                    (x['small_group']==27).sum() + 0.000001}))
    
    gr8 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s8':
                    (x['small_group']==61).sum() + 0.000001}))
    
    gr9 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s9':
                    (x['small_group']==77).sum() + 0.000001}))
    
    gr10 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s10':
                    (x['small_group']==86).sum() + 0.000001}))
    
    res = gr1.join(gr2).join(gr3).join(gr4).join(gr5).join(gr6).join(gr7).join(gr8).join(gr9).join(gr10)
    res.fillna(0, inplace=True)
    return res
    

In [ ]:
def prepare_featured_data(df):
    df['month_number'] = df['trans_date']//30
    per_month = df.groupby(['client_id', 'month_number']).agg({"amount_rur": "sum"}).groupby('client_id').agg({'amount_rur':'mean'})
    #print(per_month.head())
    large_count = df[df['amount_rur'] > 500].groupby(['client_id'])['trans_date'].agg({'large_num':'count'})
    #print(large_count.head())
    small_count = df[df['amount_rur'] < 1].groupby(['client_id'])['trans_date'].agg({'small_num':'count'})
    #print(small_count.head())
    trans_per_month = df.groupby(['client_id', 'month_number'])['amount_rur'].agg({"tran_num_mean": "count"}).groupby('client_id').agg({'tran_num_mean':'mean'})
    #print(trans_per_month.head())
    client_max = df.groupby(['client_id'])['amount_rur'].agg({"max": "max"})
    #print(client_max.head())
    res = per_month.join(client_max).join(trans_per_month).join(small_count).join(large_count)
    res.fillna(0, inplace=True)
    return res

In [228]:
from sklearn.decomposition import PCA

features = ['s1', 's2', 's3','s4', 's5','s6', 's7', 's8', 's9', 's10']
# Separating out the features
x = processed.loc[:, features].values
y = processed.index.values


pca = PCA(n_components=6)
components = pca.fit_transform(x)
pca_processed = pd.DataFrame(data = components
             , columns = ['s1', 's2', 's3','s4', 's5','s6'])
pca_processed['client_id'] = y
pca_processed.set_index('client_id', inplace=True)

print(pca_processed.head(20))

                 s1        s2        s3        s4        s5        s6
client_id                                                            
4         -0.236386  0.176500  0.042200 -0.084528 -0.029608 -0.002745
6          0.170438 -0.093678 -0.086125 -0.022603 -0.010675 -0.008973
10        -0.126935  0.085155 -0.000104 -0.063330 -0.023186 -0.004688
11         0.115190 -0.252753  0.420266 -0.025610 -0.004034 -0.017625
13         0.188682 -0.002339  0.016541 -0.054697 -0.018720 -0.008007
14        -0.221520 -0.001179 -0.032795 -0.043680 -0.012274 -0.006022
17         0.035120 -0.015058 -0.046525 -0.043534  0.030930  0.070535
19        -0.110461  0.205258  0.052837 -0.057309  0.073003 -0.005482
20         0.136331 -0.123140 -0.018508 -0.028817 -0.010196 -0.008723
25         0.092900 -0.039044 -0.050881  0.059757  0.006801 -0.007166
27         0.113123 -0.044470 -0.063369 -0.034145 -0.009134 -0.007959
28         0.029291 -0.078643 -0.074330 -0.026212 -0.011471  0.000663
31         0.641751 

In [22]:
def normalize(column_names_to_normalize, df):
    x = df[column_names_to_normalize].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    
    df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df.index)
    df[column_names_to_normalize] = df_temp
    return df

In [236]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden0 = nn.Linear(10, 8)
        self.hidden1 = nn.Linear(8, 16)
        self.hidden2 = nn.Linear(16, 32)
        
        self.bn1 = nn.BatchNorm1d(16)
        self.output = nn.Linear(32, 4)

        
    def forward(self, x):
        x = F.relu(self.hidden0(x)) 
        x = F.relu(self.bn1(self.hidden1(x))) 
        x = F.relu(self.hidden2(x))
        return F.log_softmax(self.output(x))
        

In [24]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torchvision

class Ds(Dataset):
    
    def __init__(self, data, labels):
        self.data = labels.join(data)
        print(self.data.head(30))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data.iloc[index, 1:]
        y = self.data.iloc[index, 0]

        return torch.tensor(x), torch.tensor(y)

In [ ]:
print(processed.head(10))

In [173]:
df = train_x
gr_by_client = df.groupby(['client_id'])
gr1 = gr_by_client["amount_rur"].agg({"s1": "sum"})
gr2 = gr_by_client["amount_rur"].agg({"s2": "max"}) 
gr3 = gr_by_client["amount_rur"].agg({"s3": "min"})

gr4_by_group = df.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'s4':'min'})
gr4  = gr4_by_group.loc[gr4_by_group.groupby('client_id')['s4'].idxmax()][['client_id','small_group']]
gr4.columns = ['client_id', 's4']
gr4.set_index('client_id', inplace=True)

    
gr5_by_group = df.groupby(['client_id', 'small_group'], as_index = False)['amount_rur'].agg({'s5':'sum'})
gr5  = gr5_by_group.loc[gr5_by_group.groupby('client_id')['s5'].idxmax()][['client_id','small_group']]
gr5.columns = ['client_id', 's5']
gr5.set_index('client_id', inplace=True)
    
gr6 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s6':
                    (x['small_group']==4).sum() + 0.000001}))

gr7 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s7':
                    (x['small_group']==27).sum() + 0.000001}))
    
gr8 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s8':
                    (x['small_group']==61).sum() + 0.000001}))
    
gr9 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s9':
                    (x['small_group']==77).sum() + 0.000001}))
    
gr10 = train_x.groupby(['client_id']).apply(lambda x: pd.Series({'s10':
                    (x['small_group']==86).sum() + 0.000001}))
    
res = gr1.join(gr2).join(gr3).join(gr4).join(gr5).join(gr6).join(gr7).join(gr8).join(gr9).join(gr10)
res.fillna(0, inplace=True)
print(res.head(10))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """


                  s1        s2     s3  s4  s5         s6        s7         s8  \
client_id                                                                       
4          28404.121  1341.802  0.043   2   1  93.000001  0.000001   0.000001   
6          15720.739   315.781  0.045  80   1  10.000001  0.000001   0.000001   
10         34419.365   654.893  0.045  23   1  65.000001  0.000001   0.000001   
11         26789.404  2105.058  0.388  66  88  23.000001  0.000001   2.000001   
13         17337.467  1190.322  0.043  83  11  45.000001  0.000001   0.000001   
14         26622.629  1167.291  0.043   5   1  39.000001  0.000001   1.000001   
17         58337.998  5692.843  0.028  54   1  34.000001  2.000001   9.000001   
19         46941.495  2049.338  0.259  26   1  98.000001  0.000001  21.000001   
20         79997.006  3213.706  0.043  73  13  10.000001  0.000001   0.000001   
25         77245.103  2564.078  0.647  65   1  22.000001  0.000001   4.000001   

                 s9       s

In [229]:

train_dataset = Ds(data = pca_processed, labels = train_y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  

           bins        s1        s2        s3        s4        s5        s6
client_id                                                                  
1046          0 -0.221349 -0.074568 -0.055947  0.052967 -0.006222 -0.007138
34089         2 -0.007030 -0.006213 -0.042858 -0.042787 -0.016790 -0.006694
34848         1 -0.143032  0.005832 -0.032920 -0.045435 -0.017777 -0.006340
47076         3 -0.132098  0.025297 -0.019395  0.011242 -0.018466 -0.006074
10938         2 -0.007559 -0.124109 -0.091787 -0.010345 -0.008566 -0.008870
42965         1  0.145778  0.031268 -0.019183  0.092938 -0.007810 -0.006869
21594         0 -0.079850  0.077540 -0.004460 -0.055048 -0.007715 -0.005493
7952          1  0.144882  0.028339 -0.033892 -0.050882 -0.014093 -0.006483
33810         0  0.029704  0.006959 -0.038432 -0.045973 -0.017242  0.010567
11061         2 -0.221711 -0.044365 -0.050780 -0.033735 -0.014194 -0.006460
30905         0 -0.174687 -0.061380 -0.059152 -0.022226  0.006821 -0.007909
29440       

In [234]:
train_y = ds.get_data('train_target')
#print(train_y.head())

train_x = ds.get_data('train_x')

#train_y.set_index('client_id', inplace=True)
data = prepare_featured_data_v2(train_x)
print(data.head(10))

processed = normalize(["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10"], data)

train_dataset = Ds(data = processed, labels = train_y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """


                  s1        s2     s3  s4  s5         s6        s7         s8  \
client_id                                                                       
4          28404.121  1341.802  0.043   2   1  93.000001  0.000001   0.000001   
6          15720.739   315.781  0.045  80   1  10.000001  0.000001   0.000001   
10         34419.365   654.893  0.045  23   1  65.000001  0.000001   0.000001   
11         26789.404  2105.058  0.388  66  88  23.000001  0.000001   2.000001   
13         17337.467  1190.322  0.043  83  11  45.000001  0.000001   0.000001   
14         26622.629  1167.291  0.043   5   1  39.000001  0.000001   1.000001   
17         58337.998  5692.843  0.028  54   1  34.000001  2.000001   9.000001   
19         46941.495  2049.338  0.259  26   1  98.000001  0.000001  21.000001   
20         79997.006  3213.706  0.043  73  13  10.000001  0.000001   0.000001   
25         77245.103  2564.078  0.647  65   1  22.000001  0.000001   4.000001   

                 s9       s

In [27]:
import torch 
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda"),
else:
    device = torch.device("cpu")

In [245]:
import numpy as np
import torch.optim as optim

learning_rate = 0.001


print(device)
network = Net().cuda()
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

for epoch in range(1, 9):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #if batch_idx > 2460:
            #output = network(data.cuda())
            #pred = output.data.max(1, keepdim=True)[1]
            #print('ACURACY: {}'.format((pred.flatten() == target).sum()))
       # else:
            optimizer.zero_grad()
            output = network(data.cuda())
            loss = F.cross_entropy(output, target.cuda()).cuda()
            loss.backward()
            optimizer.step()
    print('Train Epoch: {} \tLoss: {:.6f}'.format(
            epoch, loss.item()))

(device(type='cuda'),)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 	Loss: 1.211276
Train Epoch: 2 	Loss: 1.102562
Train Epoch: 3 	Loss: 1.557207
Train Epoch: 4 	Loss: 1.276192
Train Epoch: 5 	Loss: 1.266304
Train Epoch: 6 	Loss: 1.218385
Train Epoch: 7 	Loss: 1.504468
Train Epoch: 8 	Loss: 1.044336


In [1]:
print(len(res_df[res_df['large_num']>0]))

NameError: name 'res_df' is not defined

In [95]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=10, random_state=0)

# похоже где-то ошибка, возможно, где криво джоинится по айди клиента. Либо фичи полное говно
clf.fit(processed[["s1", "s2", "s3", "s4", "s5"]].values,train_y['bins'].values)
clf.score(valid_x[["s1", "s2", "s3", "s4", "s5"]].values,valid_y['bins'].values)

0.24679760888129804

In [283]:
test_y = ds.get_data('test_target')
print(ds.get_data('test_target').iloc[:, 0])
#test_y.set_index('client_id', inplace=True)
#test_x = ds.get_data('test_x')

#test_processed = normalize(["amount_rur", "max", "tran_num_mean", "small_num", "large_num"], prepare_featured_data(test_x))
#clf.score(test_processed[["amount_rur", "max", "tran_num_mean", "small_num", "large_num"]].values,test_y.values)

0        28571
1        27046
2        13240
3        19974
4        10505
5        36581
6        13137
7        29069
8        14334
9         7432
10       44221
11       40799
12       36456
13         818
14       35103
15       48997
16        4975
17        4013
18       30955
19       26778
20         709
21        9410
22       15212
23       11347
24         766
25          55
26        2493
27       46885
28       39401
29        8934
         ...  
19970     5048
19971    21095
19972    29891
19973     6069
19974    39294
19975    41915
19976    44830
19977    44923
19978    25431
19979    27672
19980    21382
19981     6221
19982    30173
19983    28339
19984    24920
19985    17978
19986    16100
19987    41833
19988    26651
19989    37973
19990    14102
19991    26308
19992      910
19993    48964
19994    13582
19995     2565
19996    31255
19997    31539
19998     4288
19999    15323
Name: client_id, Length: 20000, dtype: int64


In [274]:
print(test_y.head())

Empty DataFrame
Columns: []
Index: [28571, 27046, 13240, 19974, 10505]


In [192]:
valid_y = ds.get_data('valid_target')
#valid_y.set_index('client_id', inplace = True)

valid_x = ds.get_data('valid_x')
valid_x = normalize(["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10"], prepare_featured_data_v2(valid_x))

valid_dataset = Ds(data = valid_x, labels = valid_y)

total = len(valid_dataset)
valid_loader = DataLoader(valid_dataset, batch_size=total)  



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """


           bins        s1        s2        s3        s4        s5   s6   s7  \
client_id                                                                     
24662         2  0.014923  0.004839  0.015675  0.284211  0.066298  0.0  0.0   
47651         3  0.020765  0.003163  0.088254  0.147368  0.005525  0.0  0.0   
36224         2  0.004771  0.002299  0.092549  0.378947  0.005525  0.0  0.0   
21691         1  0.018916  0.017944  0.184883  0.173684  0.071823  0.0  0.0   
29957         3  0.025960  0.015075  0.009019  0.089474  0.011050  0.0  0.0   
30401         2  0.011060  0.002129  0.092549  0.284211  0.005525  0.0  0.0   
15639         0  0.018530  0.002320  0.068499  0.242105  0.005525  0.0  0.0   
28539         1  0.016628  0.002953  0.008804  0.378947  0.005525  0.0  0.0   
39868         3  0.038876  0.010590  0.009019  0.531579  0.132597  0.0  0.0   
48975         0  0.038527  0.008853  0.323599  0.068421  0.005525  0.0  0.0   
26720         0  0.022271  0.006457  0.006657  0.236

In [246]:
def validateNet(net):
    r, idd = iter(valid_loader).next()
    output = net(r.cuda()).cpu()
    
    pred = output.data.max(1, keepdim=True)[1]
   # print(pred)
    print('ACURACY: {}'.format((pred.flatten() == idd).sum()))
    
validateNet(network)

ACURACY: 2367


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [194]:
print(total)

5855
